### 电影评论情感分析 

无意中看到了这个数据集，该数据集包含了豆瓣上的 28 部电影的 200 万个短评，正好拿来练一练NLP。我将使用该数据集建立NLP的模型并进行测试，本实验暂时分为以下几部分：
- NN
- LSTM

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import os
import time
from tqdm import tqdm_notebook
import re
import time
import copy
import random
import jieba
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler

#### 数据预处理

首先，我们读取并预览 CSV 数据文件。

In [2]:
import pandas as pd

douban_data = pd.read_csv('DMSC.csv', index_col=0)
douban_data.head()

,Movie_Name_EN,Movie_Name_CN,Crawl_Date,Number,Username,Date,Star,Comment,Like
ID,,,,,,,,,
0,Avengers Age of Ultron,复仇者联盟2,2017-01-22,1,然潘,2015-05-13,3,连奥创都知道整容要去韩国。,2404
1,Avengers Age of Ultron,复仇者联盟2,2017-01-22,2,更深的白色,2015-04-24,2,非常失望，剧本完全敷衍了事，主线剧情没突破大家可以理解，可所有的人物都缺乏动机，正邪之间、...,1231
2,Avengers Age of Ultron,复仇者联盟2,2017-01-22,3,有意识的贱民,2015-04-26,2,2015年度最失望作品。以为面面俱到，实则画蛇添足；以为主题深刻，实则老调重弹；以为推陈出...,1052
3,Avengers Age of Ultron,复仇者联盟2,2017-01-22,4,不老的李大爷耶,2015-04-23,4,《铁人2》中勾引钢铁侠，《妇联1》中勾引鹰眼，《美队2》中勾引美国队长，在《妇联2》中终于...,1045
4,Avengers Age of Ultron,复仇者联盟2,2017-01-22,5,ZephyrO,2015-04-22,2,虽然从头打到尾，但是真的很无聊啊。,723


In [3]:
# 考虑到每个人的情感阈值存在极大差距，因此将评分分成两类，三分以下为差评
douban_data['Star']=((douban_data.Star+0.5)/3.5+1).astype(int)
douban_data.head()

,Movie_Name_EN,Movie_Name_CN,Crawl_Date,Number,Username,Date,Star,Comment,Like
ID,,,,,,,,,
0,Avengers Age of Ultron,复仇者联盟2,2017-01-22,1,然潘,2015-05-13,2,连奥创都知道整容要去韩国。,2404
1,Avengers Age of Ultron,复仇者联盟2,2017-01-22,2,更深的白色,2015-04-24,1,非常失望，剧本完全敷衍了事，主线剧情没突破大家可以理解，可所有的人物都缺乏动机，正邪之间、...,1231
2,Avengers Age of Ultron,复仇者联盟2,2017-01-22,3,有意识的贱民,2015-04-26,1,2015年度最失望作品。以为面面俱到，实则画蛇添足；以为主题深刻，实则老调重弹；以为推陈出...,1052
3,Avengers Age of Ultron,复仇者联盟2,2017-01-22,4,不老的李大爷耶,2015-04-23,2,《铁人2》中勾引钢铁侠，《妇联1》中勾引鹰眼，《美队2》中勾引美国队长，在《妇联2》中终于...,1045
4,Avengers Age of Ultron,复仇者联盟2,2017-01-22,5,ZephyrO,2015-04-22,1,虽然从头打到尾，但是真的很无聊啊。,723


数据由 9 列构成，分别是：电影英文名、中文名、爬取日期、评论 ID（从 0 开始）、用户名、发表日期、评价分数、评论内容、点赞数。然后划分数据集，随机抽取近 10000 条数据，并按电影和评分均等划分。最终实际数据为 2125032 条。

In [4]:
Movie = douban_data['Movie_Name_CN'].value_counts()
Movie

疯狂动物城     137511
大圣归来      133393
后会无期      120200
寻龙诀       113687
你的名字      113260
夏洛特烦恼     109162
釜山行       102876
爱乐之城       96620
西游伏妖篇      91452
小时代1       88903
泰囧         85677
大鱼海棠       83692
长城         83173
西游降魔篇      79962
复仇者联盟      78281
美人鱼        73882
七月与安生      68359
美国队长3      64410
变形金刚4      58746
复仇者联盟2     54153
十二生肖       46233
九层妖塔       44366
小时代3       41152
左耳         39802
湄公河行动      35093
栀子花开       30475
何以笙箫默      26797
钢铁侠1       23739
Name: Movie_Name_CN, dtype: int64

可以看到，基本都是两年前的电影，数据也是两年前爬取的，后续我们将使用利用两年前的影评训练出的模型来预测最近上映的电影。

In [5]:
sample_df = douban_data.groupby(['Movie_Name_CN', 'Star']).apply(
    lambda x: x.sample(n=int(2125056/(28*2)), replace=True, random_state=0))

sample_df.shape

(2125032, 9)

`groupby` 函数对 DataFrame 按照 `'Movie_Name_CN', 'Star'` 进行分组（两列值相同为一组），参考 [<i class="fa fa-external-link-square" aria-hidden="true"> pandas.DataFrame.groupby</i>](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)，`apply` 传入一个 `lambda` 函数，对每一组调用 `sample` 函数随机抽取出 `n` 条数据，参考 [<i class="fa fa-external-link-square" aria-hidden="true"> pandas.DataFrame.sample</i>](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html)。

接下来我们将数据划分为 80% 的训练集和 20% 的测试集，并打乱顺序。

In [6]:
from sklearn.model_selection import train_test_split


comments = sample_df.values[:, 7]
star = sample_df.values[:, 6]

x_train, x_test, y_train, y_test, = train_test_split(
    comments, star, test_size=0.2, random_state=0)

len(y_train), len(y_test), len(x_train), len(x_test)

(1700025, 425007, 1700025, 425007)

#### 分词处理

使用结巴分词，并返回分词结果和标签：

In [8]:
# 清理非中文字符，替换不需要的字符串
def clean_str(line):
    line.strip('\n')
    line = re.sub(r"[^\u4e00-\u9fff]", "", line)
    line = re.sub(
        "[0-9a-zA-Z\-\s+\.\!\/_,$%^*\(\)\+(+\"\')]+|[+——！，。？、~@#￥%……&*（）<>\[\]:：★◆【】《》;；=?？]+", "", line)
    return line.strip()


# 加载停用词
with open('stopwords.txt') as f:
    stopwords = [line.strip('\n') for line in f.readlines()]


def cut(data, labels, stopwords):
    result = []
    new_labels = []
    for index in tqdm_notebook(range(len(data))):
        comment = clean_str(data[index])
        label = labels[index]
        # 分词
        seg_list = jieba.cut(comment, cut_all=False, HMM=True)
        seg_list = [x.strip('\n')
                    for x in seg_list if x not in stopwords and len(x) > 1]
        if len(seg_list) > 1:
            result.append(seg_list)
            new_labels.append(label)
    # 返回分词结果和对应的标签
    return result, new_labels


# 分别对训练数据和测试数据分词
train_cut_result, train_labels = cut(x_train, y_train, stopwords)
test_cut_result, test_labels = cut(x_test, y_test, stopwords)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.652 seconds.
Prefix dict has been built succesfully.


#### 词向量转换

任何机器学习算法都无法直接理解词语的含义，所以我们需要将自然语言处理成算法能够理解的词向量，这里我们用到 TF-IDF 模型。TF-IDF 由两部分组成：TF（Term frequency，词频），IDF（Inverse document frequency，逆文档频率）。

TF-IDF 根据词频即词语在数据中出现的次数。总的来说，就是把一段话的每个词都配之以重要性指标，某个词在这段话的出现次数多时，它就更重要。比如一段话中提到了三次「中国」，就比另一段只提到了一次的重要。

此外，当这个词在所有话中出现的次数多时（比如「苹果」，「乔布斯」等不常见词），这个词就更加的重要。当这个词在所有的段落中出现次数都多时（比如「的」，「这」等常见词），这个词的重要性下降。对于每个词都计算这样的重要性并写成一个向量后，映射就完成了。

scikit-learn 实现了这种方法，参考 [<i class="fa fa-external-link-square" aria-hidden="true"> TfidfVectorizer</i>](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)，这是 [<i class="fa fa-external-link-square" aria-hidden="true"> CountVectorizer</i>](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)（统计词频） 和 [<i class="fa fa-external-link-square" aria-hidden="true"> TfidfTransformer</i>](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html)（转换 TF-IDF）的组合。

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TfidfVectorizer 传入原始文本
train_data = [' '.join(x) for x in train_cut_result]
test_data = [' '.join(x) for x in test_cut_result]

# max_features指定语料库中频率最高的词
n_dim = 35000

# 数据的TF-IDF信息计算
# sublinear_tf=True 时生成一个近似高斯分布的特征，可以提高大概1~2个百分点
vectorizer = TfidfVectorizer(
    max_features=n_dim, smooth_idf=True, sublinear_tf=True)

# 对训练数据训练
train_vec_data = vectorizer.fit_transform(train_data)

# 训练完成之后对测试数据转换
test_vec_data = vectorizer.transform(test_data)

/home/x/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


查看词袋模型中的前十个词语:

In [9]:
vectorizer.get_feature_names()[:10]

['一丁点', '一万', '一万个', '一万倍', '一万年', '一万次', '一万步', '一万遍', '一下下', '一下子']

至此数据预处理结束，得到了训练和测试数据的 TF-IDF 矩阵，接下来将使用 TF-IDF 矩阵进行训练和测试。

#### 模型训练

先尝试使用多层神经网络实现豆瓣电影评论分类，并通过 PyTorch 深度学习框架完成。首先，实现需要定义一些参数，例如类别数、学习率、损失函数、迭代次数等。

In [10]:
import torch.nn as nn

# 输出的类别为 2
n_categories = 2
# 学习率，请不要过大，会导致 loss 震荡
learning_rate = 0.001
# 损失函数
criterion = nn.CrossEntropyLoss()
# 迭代次数
epochs = 6
# 每次迭代同时加载的个数
batch_size = 100

在 PyTorch 中对自定义数据集都需要写一个 Dataset 进行加载数据，然后在 DataLoader 中使用。所以，第一步是定义一个自己的 Dataset 类，重写 `__getitem__` 方法，获取数据。

In [11]:
from torch.utils.data import Dataset, DataLoader

class TxtDataset(Dataset):
    def __init__(self, VectData, labels):
        # 传入初始数据，特征向量和标签
        self.VectData = VectData
        self.labels = labels

    def __getitem__(self, index):
        # DataLoader 会根据 index 获取数据
        # toarray() 是因为 VectData 是一个稀疏矩阵，如果直接使用 VectData.toarray() 占用内存太大，请勿尝试
        return self.VectData[index].toarray(), self.labels[index]-1

    def __len__(self):
        return len(self.labels)

# 线下内存足够大可以考虑增大 num_workers，并行读取数据
# 加载训练数据集
train_dataset = TxtDataset(train_vec_data, train_labels)
train_dataloader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              shuffle=True,
                              num_workers=50
                              )
# 加载测试数据集
test_dataset = TxtDataset(test_vec_data, test_labels)
test_dataloader = DataLoader(test_dataset,
                             batch_size=batch_size,
                             shuffle=False,
                             num_workers=50
                             )

定义一个神经网络，由线性层、激活函数、Dropout 依次组成。这里，我们仅仅定义了一个全连接网络。

In [14]:
class TxtModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(TxtModel, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, output_size)
        )

    def forward(self, x):
        output = self.classifier(x.double())
        return output.squeeze(1)

训练过程中，选用 Adam 作为优化器，`exp_lr_scheduler` 是为了每个步长衰减学习率。最后输出损失、准确度和执行时间。

In [15]:
# 定义模型和优化器
model = TxtModel(n_dim, 2)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 每两代衰减学习率
exp_lr_scheduler = lr_scheduler.StepLR(
    optimizer, step_size=int(epochs/2), gamma=0.1)
 
if torch.cuda.is_available():
    model = model.cuda()
    #model = nn.DataParallel(model)
    criterion = criterion.cuda()

model = model.double()

# 保存准确度最高的模型
best_model = copy.deepcopy(model)
best_accuracy = 0.0

for epoch in range(epochs):
    exp_lr_scheduler.step()
    model.train()
    loss_total = 0
    st = time.time()
    # train_dataloader 加载数据集
    for data, label in tqdm_notebook(train_dataloader):
        # 如果 GPU 可用，则使用 GPU
        if torch.cuda.is_available():
            data = data.cuda()
            label = label.cuda()
        output = model(data)
        # 计算损失
        loss = criterion(output, label)
        optimizer.zero_grad()
        # 反向传播
        loss.backward()
        optimizer.step()
        loss_total += loss.item()

    # 输出损失、训练时间等
    print('epoch {}/{}:'.format(epoch, epochs))
    print('training loss: {}, time resumed {}s'.format(
        loss_total/len(train_dataset), time.time()-st))

    model.eval()

    loss_total = 0
    st = time.time()

    correct = 0
    for data, label in test_dataloader:
        if torch.cuda.is_available():
            data = data.cuda()
            label = label.cuda()
        output = model(data)
        loss = criterion(output, label)
        loss_total += loss.item()

        _, predicted = torch.max(output.data, 1)
        correct += (predicted == label).sum().item()
    # 如果准确度取得最高，则保存准确度最高的模型
    if correct/len(test_dataset) > best_accuracy:
        best_model = copy.deepcopy(model)

    print('testing loss: {}, time resumed {}s, accuracy: {}'.format(
        loss_total/len(test_dataset), time.time()-st, correct/len(test_dataset)))


epoch 0/6:
training loss: 0.004015722215006582, time resumed 1494.8732607364655s
testing loss: 0.003235486754182074, time resumed 167.8843846321106s, accuracy: 0.8648242564804465



epoch 1/6:
training loss: 0.0027647626172974784, time resumed 1533.0914461612701s
testing loss: 0.002698652265290571, time resumed 164.81334781646729s, accuracy: 0.8897648111548143



epoch 2/6:
training loss: 0.0022649255073836257, time resumed 1530.1845548152924s
testing loss: 0.0024865395633903394, time resumed 165.37635612487793s, accuracy: 0.9011325972241193



epoch 3/6:
training loss: 0.0017764741919218477, time resumed 1537.6156966686249s
testing loss: 0.002411478553855628, time resumed 169.7466697692871s, accuracy: 0.9065070679307452



epoch 4/6:
training loss: 0.0016524011864852558, time resumed 1539.3705825805664s
testing loss: 0.0023988213636887976, time resumed 170.16531586647034s, accuracy: 0.9087482891175331



epoch 5/6:
training loss: 0.0015721819568429492, time resumed 1548.9475784301758s
testing loss: 0.002389944676247696, time resumed 170.98718237876892s, accuracy: 0.9105769472002855


跑了将近3个小时终于跑完，接下来测试模型。

#### 测试模型

接下来使用豆瓣 API 获取近期上映电影的影评，然后预测他们的情感倾向。

我们选择19年上映的流浪地球，这个国产科幻题材是我们之前的数据集里没有的类型，观众对该电影寄托的情感与之前的电影也是截然不同，这可以考验一下模型的泛化能力。

In [17]:
import json
import requests
# 26266893 为国产科幻佳作《流浪地球》，在此以《流浪地球》的影评为例
res = requests.get(
    'https://api.douban.com/v2/movie/subject/26266893/comments?apikey=0df993c66c0c636e29ecbb5344252a4a')
comments = json.loads(res.content.decode('utf-8'))['comments']
comments

[{'rating': {'max': 5, 'value': 1.0, 'min': 0},
  'useful_count': 70607,
  'author': {'uid': 'duduxiongzhifu',
   'avatar': 'https://img3.doubanio.com/icon/u2201715-15.jpg',
   'signature': '谁来拧动拧发条鸟的发条',
   'alt': 'https://www.douban.com/people/duduxiongzhifu/',
   'id': '2201715',
   'name': '嘟嘟熊之父 \U0001f9f8'},
  'subject_id': '26266893',
  'content': '还能更土更儿戏一点吗？毫无思考仅靠煽动，毫无敬畏仅余妄想。好的科幻片应该首先承认人类的无知，并跳出人类的视角去看待人与宇宙的关系，而不是一头扎入狭隘的家庭纠纷与大国情怀中自作聪明自我感动。被吹到不行的特效如同导演抡圆了膀子朝观众脸上砸各种金银珠宝，闪到不行但全无美感。有人说作为中国第一部硬科幻电影，不要跟美国比，只想说这宣传攻势票房体量已经超越大多数好莱坞商业大片了，凭啥不能和人家比？所以评价《流浪地球》很简单，你把片中所有角色换成美国人，然后再想想自己愿意打几星。',
  'created_at': '2019-01-28 22:06:27',
  'id': '1646653503'},
 {'rating': {'max': 5, 'value': 4.0, 'min': 0},
  'useful_count': 69330,
  'author': {'uid': 'tjz230',
   'avatar': 'https://img1.doubanio.com/icon/u1005928-127.jpg',
   'signature': '',
   'alt': 'https://www.douban.com/people/tjz230/',
   'id': '1005928',
   'name': '影志'},
  'subject_id': '26266893',
  'content': '电影比预期要更恢弘磅礴，晨昏线过后的永夜、火种计

In [18]:
def predict_comments(comments):
    test_comment = random.choice(comments)
# 选择其中一条分类，并去除非中文字符
    content = clean_str(test_comment['content'])
    rating = test_comment['rating']['value']
# 对评论分词
    seg_list = jieba.cut(content, cut_all=False, HMM=True)
# 去掉停用词和无意义的
    cut_content = ' '.join([x.strip('\n')
                        for x in seg_list if x not in stopwords and len(x) > 1])

# 转化为特征向量
    one_test_data = vectorizer.transform([cut_content])

# 转化为 pytorch 输入的 Tensor 数据，squeeze(0) 增加一个 batch 维度
    one_test_data = torch.from_numpy(one_test_data.toarray()).unsqueeze(0)
# 使用准确度最好的模型预测，softmax 处理输出概率，取得最大概率的下标再加 1 则为预测的标签
    pred = torch.argmax(F.softmax(best_model(one_test_data), dim=1)) + 1
    pred = pred.item()
    
    if pred==1:
        pred='差评1'
    else:
        pred='好评2'
        
    if rating<3:
        rat='差评1'
    else:
        rat='好评2'
    print('评论内容: ',content)
    print('关键字: ',cut_content)
    print('观众评价: ',rat)
    print('预测评价: ',pred)

上面依次输出了评论内容、关键字、观众评价、预测评价。接下来，我们测试基于评论预测该观众的最终评分。

In [19]:
for i in range(5):
    print('观后感: ',i)
    print(predict_comments(comments))

观后感:  0
评论内容:  充斥着模仿的痕迹某些镜头直接地心引力同款另外就是画质极渣的摄影以及播放卡顿宏大主题配以乡土气十足的人设情感以及地下小镇式场景中段的冗长更是让人疲惫倒是现象级的吴京带来现象级的吹捧狂潮惊到我了
关键字:  充斥 模仿 痕迹 镜头 地心引力 同款 画质 极渣 摄影 播放 卡顿 宏大 主题 配以 乡土气 十足 人设 情感 地下 小镇 场景 中段 冗长 更是 疲惫 现象 吴京 带来 现象 吹捧 狂潮 惊到
观众评价:  差评1
预测评价:  差评1
None
观后感:  1
评论内容:  野心远远大于能力的作品大刘小说打底导演用影视化视觉实现了大部分想象敢想敢做远超预期牛但也仅自于此了故事一塌糊涂世界观做的如此粗糙叙事完全不讲逻辑人物动机行为全都莫名其妙不能理解台词在电影里算的上数一数二的烂最恶心的是国人文化自豪情绪被名目张胆刺裸裸的利用成商业行为明明稍微用点心思就可以更好一点偏不导演编剧仿佛在对你说故事差不多得了太深了他们看不懂就是要俗气直白最要紧的是中国人要拯救地球吴京要操翻世界联合政府中国就爱看这个中国人就只配看这样的科幻故事对啊看看这些评论就只配看这样的科幻故事白瞎了这样的制作白瞎了中国人自己的硬科团队
关键字:  野心 远远 大于 能力 作品 小说 打底 导演 影视 视觉 大部分 想象 远超 预期 自于 故事 一塌糊涂 世界观 粗糙 叙事 逻辑 人物 动机 莫名其妙 理解 台词 电影 里算 数一数二 恶心 国人 文化 自豪 情绪 名目 张胆 利用 商业行为 明明 稍微 用点 心思 更好 一点 导演 编剧 仿佛 故事 太深 看不懂 俗气 直白 要紧 中国 人要 拯救 地球 吴京要 操翻 世界 联合政府 中国 爱看 中国 只配 科幻 故事 评论 只配 科幻 故事 制作 中国 硬科 团队
观众评价:  差评1
预测评价:  差评1
None
观后感:  2
评论内容:  中国导演能拍出这样的硬科幻看到这样的完成度想想就激动可以让人原谅一切不完美说这部电影是中国科幻电影的元年自然是站不住脚毕竟国内早就拍过很多软科幻但说流浪地球是中国硬科幻电影的第一座里程碑或者说中国硬科幻电影的元年这是任何人都无可反驳的它的工业化程度在国内绝对是前所未有的是一部可以载入中国电影史的电影其实中国观众对于这类型的硬科幻并不陌生只是国内一直没有拍摄这种硬科幻大制作的

#### 可以看到，我们使用17年的影评来预测19年新上映的电影效果也是非常不错，我们随机抽取的这几条影评全部预测正确，这说明我们的模型虽然很简单，但是泛化能力还不错。